In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
import sympy as sp

In [8]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('hundo_equations.csv')

# Initialize lists to hold function details and the number of variables
functions = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# parameters']
    
    function_details = {
        'formula': formula,
        'num_variables': num_vars,
        'parameters': []
    }
    
    for i in range(1, num_vars + 1):
        param_name = row.get(f'p{i}_name', None)
        if pd.notna(param_name):
            function_details['parameters'].append({
                'name': param_name
            })
    
    functions.append(function_details)

In [9]:
functions

[{'formula': 'ax^2 + bx + c',
  'num_variables': 3,
  'parameters': [{'name': 'a'}, {'name': 'b'}, {'name': 'c'}]},
 {'formula': 'a e^(bx)',
  'num_variables': 2,
  'parameters': [{'name': 'a'}, {'name': 'b'}]},
 {'formula': 'ax^3 + bx^2 + cx',
  'num_variables': 3,
  'parameters': [{'name': 'a'}, {'name': 'b'}, {'name': 'c'}]},
 {'formula': 'a/(x + b)',
  'num_variables': 2,
  'parameters': [{'name': 'a'}, {'name': 'b'}]},
 {'formula': 'a sqrt(x) + b',
  'num_variables': 2,
  'parameters': [{'name': 'a'}, {'name': 'b'}]},
 {'formula': 'a ln(bx + c)',
  'num_variables': 3,
  'parameters': [{'name': 'a'}, {'name': 'b'}, {'name': 'c'}]},
 {'formula': 'a x^2 ln(bx)',
  'num_variables': 2,
  'parameters': [{'name': 'a'}, {'name': 'b'}]},
 {'formula': 'ax^3 + b',
  'num_variables': 2,
  'parameters': [{'name': 'a'}, {'name': 'b'}]},
 {'formula': 'a x/(b + c x)',
  'num_variables': 3,
  'parameters': [{'name': 'a'}, {'name': 'b'}, {'name': 'c'}]},
 {'formula': 'a/(x^2 + b)',
  'num_variables